In [6]:
import numpy as np
import scipy.linalg as la
#from numpy import linalg as LA
import math
import cmath

def decToBin (dec, L):
    return bin(dec).replace('0b','').zfill(L)

def binToDec (bin):
    return int(bin, 2)

def hamiltonian (L, J, delta):
    #print("M: ", M)
    H = np.zeros((2**L, 2**L))
    for i in range(2**L):
        basis = [decToBin(k,L) for k in range(2**L)]
        for j in range(L-1):
            s1 = int(basis[i][j])-0.5
            s2 = int(basis[i][j+1])-0.5
            H[i,i] = H[i,i] + delta*s1*s2 #diagonal part, H_p
            if (s1 == -0.5 and s2 == 0.5): #s_plus_s_minus
                m = binToDec(basis[i][:j] + '10' + basis[i][(j+2):])
                H[m, i] += 0.5*J
                
            if (s1 == 0.5 and s2 == -0.5): #s_minus_s_plus
                k = binToDec(basis[i][:j] + '01' + basis[i][j+2:])
                H[k, i] += 0.5*J
    return H

def printEigen (res, new, L):
    basis = [decToBin(k,L) for k in range(2**L)]
    for i in range(len(res[0])):
        eigen = str(round(res[0][i], 6)) + ' -> '
        for j in range(len(res[1][i])):
            eigen += str(round(res[1][i][j], 6)) + ' * ' + str(basis[new[j]]) + ' + '
        print (eigen)
        
def groundStateEnergy (hamiltonian):
    eig = la.eigh(H)
    eigVal = eig [0]
    return np.min(eigVal)

In [7]:
L = 8
J = 1
delta = 1
H = hamiltonian (L, J, delta)
ED0 = min (la.eigh(H)[0])
print(ED0)
ener = la.eigh(H)[0]
#for e in ener:
    #    print(45, e)

-3.374932598687889


In [8]:
# Lanczos
import random

lanSteps = 41

phiRand = np.random.rand(2**L) - np.full(2**L,0.5)
phiRandNorm = np.dot(phiRand, phiRand)
phi0 = phiRand/math.sqrt(phiRandNorm)
print (np.dot(phi0, phi0))

lanH = np.zeros((lanSteps, lanSteps))
lanEner = np.zeros(lanSteps)

0.9999999999999998


In [14]:
a = np.dot(phi0, np.dot(H, phi0))
lanH[0][0] = a
#print (min (la.eigh([[a]])[0]))
tmp = np.dot(H, phi0)
tmp2 = tmp - a*phi0
b = math.sqrt(np.dot(tmp2, tmp2))
lanH [1][0] = b
lanH [0][1] = b
phi1 = tmp2/b
tempH = lanH [:2, :2]

curr = phi1
prev = phi0


In [18]:
for step in range(1, lanSteps - 1):
    tmp = np.dot(H, curr)
    a = np.dot(curr, np.dot(H, curr))
    lanH [step][step] = a
    tmp2 = tmp - a*curr - b*prev
    b = math.sqrt(np.dot(tmp2, tmp2))
    lanH [step][step + 1] = b
    lanH [step + 1][step] = b
    tempH = lanH [:(step +1), :(step +1)]
   # print(step, tempH)
    e0 = min (la.eigh(tempH)[0])
    lanEner [step] = e0
    print (step,'\t', np.abs((e0 - ED0)/ED0))    
    prev = curr
    curr = tmp2/b
    

1 	 0.45971130678509176
2 	 0.15305224782416876
3 	 0.09528833626433551
4 	 0.08424018884291148
5 	 0.08413912697553634
6 	 0.08413709559845489
7 	 0.08413701441187768
8 	 0.08413701175038198
9 	 0.08413701146769559
10 	 0.08413701126125696
11 	 0.08413701125449535
12 	 0.08413701125423508
13 	 0.08413701125417113
14 	 0.08413701125416323
15 	 0.08413701125416086
16 	 0.08413701125414876
17 	 0.08413701125414508
18 	 0.08413701125414402
19 	 0.08413701125414481
20 	 0.08413701125414613
21 	 0.08413701125414534
22 	 0.08413701125414626
23 	 0.08413701125414613
24 	 0.08413701125414613
25 	 0.08413701125414691
26 	 0.08413701125414638
27 	 0.08413701125414534
28 	 0.08413701125414508
29 	 0.005554616881545502
30 	 0.0006894182837533696
31 	 9.353666204746289e-05
32 	 1.681213445848789e-06
33 	 2.592563633329471e-08
34 	 6.77725248596046e-10
35 	 2.6331923835897002e-11
36 	 2.436683918132368e-12
37 	 2.686957857666214e-13
38 	 3.158030782761466e-15
39 	 1.579015391380733e-15
